In [38]:
from utils import *
import rich
from pydantic import BaseModel, Field
from typing import Literal, List, Optional
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import OutputFixingParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [39]:
load_dotenv()

True

In [40]:
file_list = get_file_paths('output', file_pattern='.txt')
dfs = [pd.read_csv(file, sep='\t') for file in file_list]
dfs = [df.drop_duplicates(subset=["description"]) for df in dfs]
df = pd.concat(dfs, ignore_index=True)
len(df)

14347

In [41]:
df = df.drop_duplicates(subset=["description"])
len(df)

6113

In [42]:
df['pubDate'] = pd.to_datetime(df['pubDate'])
df['pubDate'] = df['pubDate'].dt.strftime('%Y-%m-%d')


In [43]:
df

,link,guid,type,id,sponsored,title,description,pubDate
0,https://www.cnbc.com/2025/07/30/microsoft-mark...,108179389,cnbcnewsstory,108179389,False,Microsoft tops $4 trillion in market cap after...,"Based on its post-market trading, Microsoft ha...",2025-07-30
1,https://www.cnbc.com/2025/07/30/metas-reality-...,108178021,cnbcnewsstory,108178021,False,Meta’s Reality Labs posts $4.53 billion loss i...,Meta’s Reality Labs posts $4.53 billion loss i...,2025-07-30
2,https://www.cnbc.com/2025/07/25/india-under-pr...,108171329,cnbcnewsstory,108171329,False,Trump has slapped steep tariffs on India. Here...,U.S. President Donald Trump on Wednesday annou...,2025-07-31
3,https://www.cnbc.com/2025/07/31/trumps-aug-1-t...,108178228,cnbcnewsstory,108178228,False,Trump’s tariff deadline is near. Here’s a look...,The U.S. has managed to make only eight deals ...,2025-07-31
4,https://www.cnbc.com/2025/07/30/fed-leaves-int...,108179045,cnbcnewsstory,108179045,False,"Divided Fed holds key interest rate steady, de...",A divided Federal Reserve on Wednesday voted t...,2025-07-30
...,...,...,...,...,...,...,...,...
14282,https://www.cnbc.com/2026/01/23/india-live-ent...,108249766,cnbcnewsstory,108249766,False,India’s youth are turning concerts into an eco...,India saw a 17% rise in the live events space ...,2026-01-23
14283,https://www.cnbc.com/2026/01/23/japan-december...,108255438,cnbcnewsstory,108255438,False,"Japan inflation cools to 2.1%, lowest since Ma...",Core inflation touched its lowest level since ...,2026-01-23
14284,https://www.cnbc.com/2026/01/22/musk-says-tesl...,108255973,cnbcnewsstory,108255973,False,Elon Musk says Tesla taking safety supervisors...,Tesla CEO Elon Musk said on X that his EV comp...,2026-01-23
14285,https://www.cnbc.com/2026/01/22/intel-intc-ear...,108255813,cnbcnewsstory,108255813,False,"Intel stock plunges 13% on soft guidance, conc...",Intel reported fourth-quarter earnings Thursda...,2026-01-22


In [44]:
dfs = [g.copy() for _, g in df.groupby('pubDate')]
len(dfs)

186

In [45]:
dfs = [ df.assign(description=df['title'] + ': ' + df['description']) for df in dfs ]
dfs = [ df.assign(description=df['pubDate'] + ': ' + df['description']) for df in dfs ]
docs = [df_to_docs(df, content_column='description', metadata_columns=['link', 'guid', 'type', 'id', 'sponsored', 'pubDate']) for df in dfs]
len(docs)


186

In [22]:
# docs = [doc[:5] for doc in docs]
# len(docs)

In [46]:
joined_contents = [".".join(doc.page_content for doc in sublist) for sublist in docs]
# len(joined_contents)
joined_contents = joined_contents[:5]

In [26]:
def extract_info_from_rss(all_text):
    # all_text = ". ".join(doc.page_content for doc in docs)

    # Define a rich persona in the system message with added expertise
    system_message = SystemMessagePromptTemplate.from_template(
        """You are Ava, a sharp and insightful trader assistant of a hedge fund.
    You provide clear, concise, and actionable insights based on news feeds focusing on sectors rotation.
    In addition, for each sector provide outlook, primary catalyst, and trading insights.
    In addition, Describe more pro-cyclical and geopolitical shift if any.
    In addition, list bullish and bearish sectors/companies to look at into 2026.
    In addition, provide TACO (Trump Always Chickens Out) trade if any.
    Maintain a friendly, confident, and professional tone, making complex concepts accessible and useful."""
    )

    # Define the human message template with instructions and JSON schema
    human_message = HumanMessagePromptTemplate.from_template(
        """Extract the information required from the news feed below.


    News feed:
    {news_feed}

    """
    )
    # Create the chat prompt template
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])


    llm = ChatOpenAI(temperature=0)
    chain = chat_prompt | llm | StrOutputParser()

    result = chain.invoke({"news_feed": all_text})
    return(result)

In [27]:
infos = [extract_info_from_rss(doc) for doc in joined_contents]
[rich.print(info) for info in infos]

Based on the news feed provided, it seems to suggest a cautionary note regarding consumers being enticed by 
ultra-cheap deals. This could imply potential risks or hidden costs associated with such deals that may not be 
immediately apparent to consumers. As a trader assistant focusing on sector rotation, this could indicate a 
potential impact on consumer discretionary sectors.

Outlook:
Consumer discretionary sector may face challenges as consumers navigate the trade-offs of ultra-cheap deals, 
potentially impacting consumer spending habits and company revenues in this sector.

Primary Catalyst:
The primary catalyst could be the allure of ultra-cheap deals leading to shifts in consumer behavior and 
preferences, influencing the performance of companies within the consumer discretionary sector.

Trading Insights:
- Monitor consumer sentiment and spending patterns closely to gauge the impact on the consumer discretionary 
sector.
- Consider diversifying investments across sectors less affected by potential changes in consumer behavior.
- Stay informed about any regulatory changes or market trends that could further impact consumer spending.

Pro-Cyclical Shift:
A pro-cyclical shift may occur if consumers prioritize saving over spending due to concerns about hidden costs or 
risks associated with ultra-cheap deals. This could lead to a slowdown in economic growth and impact sectors 
dependent on consumer spending.

Geopolitical Shift:
Geopolitical factors such as trade agreements, tariffs, or global economic conditions could also influence consumer
behavior and sector performance, adding another layer of complexity to market dynamics.

Bullish Sectors/Companies:
- Technology companies offering innovative solutions.
- Healthcare sector with a focus on biotech and pharmaceuticals.

Bearish Sectors/Companies:
- Consumer discretionary companies heavily reliant on consumer spending.
- Energy sector facing challenges from regulatory changes and market volatility.

TACO (Trump Always Chickens Out) Trade:
Consider hedging against potential market volatility by diversifying investments across defensive sectors such as 
utilities or consumer staples, which tend to be more resilient during uncertain times.

From the news feed provided:

1. Geopolitical Shift:
   - Trump is expected to show no mercy to either a lame duck Ishiba or because of a political vacuum, as mentioned
by advisory firm Quantum Strategy. This indicates potential political instability or power struggles in the near 
future.

2. Pro-Cyclical Shift:
   - China's micro drama market surpassing the country's box-office revenue for the first time in 2024 suggests a 
shift towards more entertainment consumption, which could indicate a pro-cyclical trend in the entertainment 
sector.

3. Sector Rotation Insights:
   - Outlook: The EV sector is gaining traction with advancements in sustainable driving technology for safer, 
cheaper, and more powerful batteries.
   - Primary Catalyst: Technological advancements in sustainable driving technology.
   - Trading Insights: Consider investing in companies involved in EV battery technology or sustainable driving 
solutions.

4. Bullish Sectors/Companies for 2026:
   - EV sector: Companies like Tesla, BYD, and Panasonic involved in EV battery technology.
   - Entertainment sector: Companies in the micro drama market in China or global entertainment companies.

5. Bearish Sectors/Companies for 2026:
   - Traditional automotive sector: Companies heavily reliant on internal combustion engine vehicles.
   - Companies in sectors facing regulatory challenges or lacking innovation.

6. TACO (Trump Always Chickens Out) Trade:
   - Monitor the political developments related to Trump's actions and statements, as they could impact market 
sentiment and certain sectors. Consider adjusting positions based on any significant shifts in policies or 
geopolitical events related to Trump.

[None, None]

In [28]:
def extract_info_from_rss(all_text):

    # Define the Pydantic model for structured output
    class Sector(BaseModel):
        Date: str = Field(..., description='The publication date of the news')
        Name: Literal[
            "Commercial Services",
            "Communications",
            "Consumer Durables",
            "Consumer Non-Durables",
            "Consumer Services",
            "Distribution Services",
            "Electronic Technology",
            "Energy Minerals",
            "Finance",
            "Health Services",
            "Health Technology",
            "Industrial Services",
            "Non-Energy Minerals",
            "Process Industries",
            "Producer Manufacturing",
            "Retail Trade",
            "Technology Services",
            "Transportation",
            "Utilities"
            ] = Field(..., description="Name of the sector")
        Outlook: str = Field(..., description='A sector outlook describes the expected future performance and conditions based on data, trends, and risks.')
        Catalyst: str = Field(..., description="A primary catalyst of a sector is the single most influential force that is expected to drive major change—positive or negative—across an entire industry.")
        Trading_insights: str = Field(..., description="Practical, actionable interpretation of the sector’s expected future conditions—something that can guide investment or trading decisions.")
        Direction_momentum: Literal["Strength", "Weakness", "Volatility", "Rotation in favor", "Rotation out of favor"] = Field(..., description="The direction of momentum of a sector is the prevailing trend in how that sector’s prices, performance, and investor sentiment are moving over a given period.")
        Sector_vs_market_position: Literal['Leading', 'Lagging', 'In Line'] = Field(..., description="Relative positioning of a sector to the overall market is the sector’s performance and strength compared to the broader market, showing whether it is leading, lagging, or moving in line with the market trend.")
    
    
    # Create the base JSON output parser
    base_parser = JsonOutputParser(pydantic_object=Sector)

    # Create the output-fixing parser wrapping the base parser and using an LLM to fix errors
    llm_for_fixing = ChatOpenAI(temperature=0)
    fixing_parser = OutputFixingParser.from_llm(parser=base_parser, llm=llm_for_fixing)

    # Define a rich persona in the system message with added expertise
    system_message = SystemMessagePromptTemplate.from_template(
        """You are Ava, a sharp and insightful trader assistant skilled is deriving sector trading insights from news."""
    )

    # Define the human message template with instructions and JSON schema
    human_message = HumanMessagePromptTemplate.from_template(
        """Extract information required.


    News feed:
    {news_feed}

    """
    )

    # Create the chat prompt template
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

    # Compose the chain: prompt -> LLM -> fixing parser
    llm = ChatOpenAI(temperature=0)
    chain = chat_prompt | llm | fixing_parser
    # all_text = ". ".join(doc.page_content for doc in docs)
    result = chain.invoke({"news_feed": all_text})
    return result

# extract_info_from_rss(joined_contents[0])



In [47]:
joined_contents

['2025-07-21: Chinese companies are waging fierce price wars — but the consumer isn’t always the winner: Though consumers may be lured by ultra-cheap deals, the trade-offs for them are more complicated than they might seem.',
 '2025-07-22: Japan could be in a weaker position in trade talks after Ishiba\'s upper house defeat, experts say:  \u2060\\Trump will show no mercy to [either] a lame duck Ishiba or because of a political vacuum,\\" advisory firm Quantum Strategy said. ".2025-07-22: How China’s $7 billion micro drama industry is taking on the U.S. entertainment industry: China\'s micro drama market surpassed the country\'s box-office revenue for the first time in 2024..2025-07-22: China\'s affluent are as pessimistic about the economy as they were during the Covid-19 pandemic: A study of affluent Chinese by Oliver Wyman found 22% of respondents were negative about the economy when surveyed in May, just above the 21% in October 2022..2025-07-22: The race to roll out solid-state bat

In [48]:
[rich.print(doc) for doc in joined_contents]

2025-07-21: Chinese companies are waging fierce price wars — but the consumer isn’t always the winner: Though 
consumers may be lured by ultra-cheap deals, the trade-offs for them are more complicated than they might seem.

2025-07-22: Japan could be in a weaker position in trade talks after Ishiba's upper house defeat, experts say:  
⁠\Trump will show no mercy to  a lame duck Ishiba or because of a political vacuum,\" advisory firm Quantum Strategy
said. ".2025-07-22: How China’s $7 billion micro drama industry is taking on the U.S. entertainment industry: 
China's micro drama market surpassed the country's box-office revenue for the first time in 2024..2025-07-22: 
China's affluent are as pessimistic about the economy as they were during the Covid-19 pandemic: A study of 
affluent Chinese by Oliver Wyman found 22% of respondents were negative about the economy when surveyed in May, 
just above the 21% in October 2022..2025-07-22: The race to roll out solid-state batteries is picking up steam 
again: Sometimes referred to as the \holy grail\" of sustainable driving, proponents say the tech offers safer, 
cheaper and more powerful batteries for EVs."

2025-07-23: CNBC's The China Connection newsletter: Beijing has more to worry about than just deflation: The key 
issue, economists say, is boosting productivity, something that requires a change in government 
incentives..2025-07-23: Singapore inflation remains at over four-year low ahead of monetary policy decision: The 
inflation figure was lower than the 0.9% expected by economists, and comes ahead of the country's monetary policy 
decision later in July..2025-07-23: China's $167 billion Tibetan mega-dam is set to boost these hydro-equipment and
materials' stocks : Dongfang Electric and Tibet Tianlu are among analysts' top picks as the mega-dam project will 
likely prompt a surge in demand for hydropower equipment, industrial materials. .2025-07-23: Japan's Prime Minister
Ishiba denies reports suggesting he could resign in August: Reuters : The reports come after Trump announced a 
trade deal with Japan, and days after Ishiba's governing coalition lost its majority in the country's upper house. 
.2025-07-23: The most powerful passports of 2025: The US and the UK fall, while India rises: The small nation-state
of Singapore again tops Henley Passport Index, followed by Japan and South Korea which are tied for second place. 
.2025-07-23: New entrance fee to visit Europe set to triple, says European Commission : The price for an ETIAS 
travel authorization to enter many parts of Europe is set to jump from 7 euros ($8) to 20 euros ($23), the European
Commission said Friday..2025-07-23: Europe’s most valuable firm SAP flags U.S. trade slowdown but says Japan deal 
gives 'hope': Deal-signing at the German cloud computing and software giant SAP turned cautious in the second 
quarter, CFO Dominik Asam told CNBC.

2025-07-24: CNBC's Inside India newsletter: Leaving, but not letting go — India’s wealthy move abroad, but stay 
invested: It is crucial for New Delhi to implement systemic changes to make India an attractive place to live and 
invest in, analysts said..2025-07-24: EU chief says China trade must become more balanced: Xi's meeting with EU 
leaders come as the 25th EU-China summit gets under way.2025-07-24: China plans subsidy vouchers for seniors to 
ease strain on its aging population, drive consumption : China plans to offer vouchers to elderly residents in a 
rare use of direct fiscal aid to tackle its rapidly aging population and strained social welfare 
system..2025-07-24: Trump to visit Federal Reserve, escalating campaign against Chair Powell: This is the first 
time in nearly two decades that an American president will be making an official trip to the central 
bank..2025-07-24: South Korea avoids technical recession as GDP expands 0.6% in second quarter: This was higher 
than the 0.5% expected by economists polled by Reuters, and a reversal from the 0.2% contraction seen in the first 
quarter..2025-07-24: Nvidia supplier SK Hynix posts record second-quarter profit and revenue on strong AI memory 
demand  : Revenue rose over 35% in the June quarter compared with the same period a year earlier, while operating 
profit jumped nearly 69%, year on year..2025-07-24: Tesla loses market share in Europe for sixth straight month as 
competition grows: It wasn't just Tesla that reported a drop in new car registrations in June, however. Europe's 
four best-selling automakers all sold fewer cars..2025-07-24: Deutsche Bank shares gain 9% after second-quarter 
profit beat: Across the board, Deutsche Bank noted an impact from the relative strength of the euro against the 
U.S. dollar..2025-07-24: World’s largest olive oil producer warns U.S. consumers of a double whammy from Trump 
tariffs: The European Union is the leading producer, consumer and exporter of olive oil.

2025-07-25: Thailand-Cambodia border clashes: Cambodia's economy has more to lose, analysts say: \Unlike Thailand, 
Cambodia has fewer policy tools ... to cushion the blow,\" says Sreeparna Banerjee, associate research fellow at 
Observer Research Foundation.".2025-07-25: Textiles to whisky: U.K.–India 'historic' deal is set to boost bilateral
trade by over $34 billion a year: The deal was signed in the presence of Indian Prime Minister Narendra Modi and 
his UK counterpart, Keir Starmer..2025-07-25: Europe's companies sound alarm over tariff hits amid scramble to 
strike U.S. trade deal: Several European firms have downgraded their guidance due to the impact of White House 
trade policies. .2025-07-25: Volkswagen cuts guidance after taking $1.5 billion hit from U.S. tariffs in first 
half: Europe's biggest carmaker posted a sharp drop in second-quarter profit, primarily due to high costs from 
increased U.S. import tariffs.

[None, None, None, None, None]

In [49]:
infos = [extract_info_from_rss(doc) for doc in joined_contents]
[rich.print(info) for info in infos]

{
    'Date': '2022-10-20',
    'Name': 'Retail Trade',
    'Outlook': 'The retail sector in China is experiencing intense price competition leading to price wars among 
companies.',
    'Catalyst': 'Intense price competition among Chinese companies',
    'Trading_insights': 'Despite the allure of low prices, there are trade-offs that may not benefit consumers in 
the long run.',
    'Direction_momentum': 'Weakness',
    'Sector_vs_market_position': 'Lagging'
}

{
    'Date': '2024-07-15',
    'Name': 'Technology Services',
    'Outlook': 'Positive',
    'Catalyst': "Growth of China's micro drama industry",
    'Trading_insights': 'Consider implications on media, technology, and consumer behavior sectors',
    'Direction_momentum': 'Strength',
    'Sector_vs_market_position': 'Leading'
}

{
    'Date': '2025-07-28',
    'Name': 'Technology Services',
    'Outlook': 'Positive',
    'Catalyst': 'Trade deal with Japan',
    'Trading_insights': 'Cautious optimism due to trade deal',
    'Direction_momentum': 'Strength',
    'Sector_vs_market_position': 'Leading'
}

{
    'Date': '2022-01-01',
    'Name': 'Finance',
    'Outlook': 'Positive outlook for the finance sector with expected growth in investments and market 
performance.',
    'Catalyst': 'Upcoming regulatory changes that are anticipated to boost financial sector activities.',
    'Trading_insights': 'Investors are advised to consider long-term positions in finance sector stocks.',
    'Direction_momentum': 'Strength',
    'Sector_vs_market_position': 'Leading'
}

1

[None, None, None, None, None]

In [58]:
def extract_info_from_rss(all_text):



    # Keep your existing Sector model unchanged
    class Sector(BaseModel):
        Date: str = Field(..., description='The publication date of the news')
        Name: Literal[
            "Commercial Services",
            "Communications",
            "Consumer Durables",
            "Consumer Non-Durables",
            "Consumer Services",
            "Distribution Services",
            "Electronic Technology",
            "Energy Minerals",
            "Finance",
            "Health Services",
            "Health Technology",
            "Industrial Services",
            "Non-Energy Minerals",
            "Process Industries",
            "Producer Manufacturing",
            "Retail Trade",
            "Technology Services",
            "Transportation",
            "Utilities"
            ] = Field(..., description="Name of the sector")
        Outlook: str = Field(..., description='A sector outlook describes the expected future performance and conditions based on data, trends, and risks.')
        Catalyst: str = Field(..., description="A primary catalyst of a sector is the single most influential force that is expected to drive major change—positive or negative—across an entire industry.")
        Trading_insights: str = Field(..., description="Practical, actionable interpretation of the sector's expected future conditions—something that can guide investment or trading decisions.")
        Direction_momentum: Literal["Strength", "Weakness", "Volatility", "Rotation in favor", "Rotation out of favor"] = Field(..., description="The direction of momentum of a sector is the prevailing trend in how that sector's prices, performance, and investor sentiment are moving over a given period.")
        Sector_vs_market_position: Literal['Leading', 'Lagging', 'In Line'] = Field(..., description="Relative positioning of a sector to the overall market is the sector's performance and strength compared to the broader market, showing whether it is leading, lagging, or moving in line with the market trend.")

    # New wrapper model for multiple sectors
    class MultiSectorAnalysis(BaseModel):
        """Analysis of multiple sectors from news feed."""
        sectors: List[Sector] = Field(
            ...,
            description="List of ALL relevant sectors found in the news (1-8 max). Only include sectors with clear evidence.",
            default_factory=list
        )
    
    # 1. Create messages WITHOUT partial first
    system_message = SystemMessagePromptTemplate.from_template(
        """You are Ava, a sharp trader assistant. Analyze the FULL news feed and extract **ALL relevant sectors** mentioned.

    CRITICAL RULES:
    - Output ONLY valid JSON matching this schema: {schema}
    - If NO sector: use empty [] 
    - Each sector must use a Name from the enum
    - Fill ALL required fields for each sector
    - No duplicate sectors
    - No extra text outside JSON"""
    )

    human_message = HumanMessagePromptTemplate.from_template(
        """News feed:
    {news_feed}"""
    )

    # 2. Create ChatPromptTemplate
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

    # 3. NOW partial with schema (after creation)
    base_parser = JsonOutputParser(pydantic_object=MultiSectorAnalysis)
    chat_prompt = chat_prompt.partial(schema=base_parser.get_format_instructions())

    # 4. Build chain
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    chain = chat_prompt | llm.with_structured_output(MultiSectorAnalysis)

    # Invoke
    result = chain.invoke({"news_feed": all_text})
    # results = [chain.invoke({"news_feed": doc}) for doc in docs[:5]]
    return result



In [59]:
infos = [extract_info_from_rss(doc) for doc in joined_contents]
[rich.print(info) for info in infos]

MultiSectorAnalysis(
    sectors=[
        Sector(
            Date='2025-07-21',
            Name='Consumer Services',
            Outlook='The consumer services sector is facing challenges due to aggressive pricing strategies that 
may not benefit consumers in the long run.',
            Catalyst='Intense price competition among Chinese companies.',
            Trading_insights='Investors should be cautious as the price wars may lead to reduced service quality 
and long-term consumer dissatisfaction.',
            Direction_momentum='Weakness',
            Sector_vs_market_position='Lagging'
        )
    ]
)

MultiSectorAnalysis(
    sectors=[
        Sector(
            Date='2025-07-22',
            Name='Consumer Services',
            Outlook='The micro drama industry in China is growing rapidly, indicating a positive trend in consumer 
entertainment preferences.',
            Catalyst="The rise of China's micro drama market surpassing traditional box-office revenue.",
            Trading_insights='Investors may consider opportunities in the entertainment sector, particularly in 
micro dramas, as they gain popularity.',
            Direction_momentum='Strength',
            Sector_vs_market_position='Leading'
        ),
        Sector(
            Date='2025-07-22',
            Name='Electronic Technology',
            Outlook='The development of solid-state batteries is gaining momentum, which could revolutionize the 
electric vehicle market.',
            Catalyst='Advancements in solid-state battery technology are seen as a major breakthrough for 
sustainable driving.',
            Trading_insights='Investors should monitor companies involved in battery technology as this sector is 
poised for growth.',
            Direction_momentum='Strength',
            Sector_vs_market_position='Leading'
        )
    ]
)

MultiSectorAnalysis(
    sectors=[
        Sector(
            Date='2025-07-23',
            Name='Energy Minerals',
            Outlook='Positive outlook due to increased demand for hydropower equipment and industrial materials 
from the Tibetan mega-dam project.',
            Catalyst="China's $167 billion Tibetan mega-dam project is expected to drive demand for hydro-equipment
and materials.",
            Trading_insights='Investors may consider stocks like Dongfang Electric and Tibet Tianlu as potential 
beneficiaries of the increased demand.',
            Direction_momentum='Strength',
            Sector_vs_market_position='Leading'
        ),
        Sector(
            Date='2025-07-23',
            Name='Industrial Services',
            Outlook='Positive outlook as the Tibetan mega-dam project boosts demand for industrial materials.',
            Catalyst='The construction of the Tibetan mega-dam is a significant catalyst for the industrial 
services sector.',
            Trading_insights='Focus on companies involved in providing materials and services for large 
infrastructure projects.',
            Direction_momentum='Strength',
            Sector_vs_market_position='Leading'
        ),
        Sector(
            Date='2025-07-23',
            Name='Technology Services',
            Outlook='Cautious outlook due to trade slowdown in the U.S. impacting deal-signing.',
            Catalyst='U.S. trade slowdown is affecting the performance of technology service firms like SAP.',
            Trading_insights="Investors should monitor SAP's performance and the impact of international trade 
agreements on technology services.",
            Direction_momentum='Weakness',
            Sector_vs_market_position='Lagging'
        )
    ]
)

MultiSectorAnalysis(
    sectors=[
        Sector(
            Date='2025-07-24',
            Name='Finance',
            Outlook='Positive outlook due to strong performance and profit beats in major banks like Deutsche 
Bank.',
            Catalyst='Strong euro against the U.S. dollar impacting bank profits positively.',
            Trading_insights='Investors may consider increasing exposure to European banks given the recent profit 
growth.',
            Direction_momentum='Strength',
            Sector_vs_market_position='Leading'
        ),
        Sector(
            Date='2025-07-24',
            Name='Consumer Non-Durables',
            Outlook='Challenging outlook due to tariffs impacting olive oil prices.',
            Catalyst='Trump tariffs affecting the olive oil market and consumer prices.',
            Trading_insights='Investors should monitor the impact of tariffs on consumer goods and consider 
alternatives.',
            Direction_momentum='Weakness',
            Sector_vs_market_position='Lagging'
        ),
        Sector(
            Date='2025-07-24',
            Name='Electronic Technology',
            Outlook='Strong outlook driven by demand for AI-related products.',
            Catalyst='Increased demand for AI memory products boosting profits for companies like SK Hynix.',
            Trading_insights='Investors may want to focus on tech companies involved in AI and memory production.',
            Direction_momentum='Strength',
            Sector_vs_market_position='Leading'
        )
    ]
)

MultiSectorAnalysis(
    sectors=[
        Sector(
            Date='2025-07-25',
            Name='Consumer Durables',
            Outlook='Challenging due to increased costs and tariffs affecting profitability.',
            Catalyst='U.S. trade policies impacting import tariffs.',
            Trading_insights='Investors should be cautious as companies like Volkswagen are downgrading guidance 
due to tariff impacts.',
            Direction_momentum='Weakness',
            Sector_vs_market_position='Lagging'
        ),
        Sector(
            Date='2025-07-25',
            Name='Commercial Services',
            Outlook='Potential growth from international trade agreements, but risks from tariffs remain.',
            Catalyst='U.K.-India trade deal expected to boost bilateral trade significantly.',
            Trading_insights='Monitor developments in trade agreements as they may provide opportunities for 
growth.',
            Direction_momentum='Strength',
            Sector_vs_market_position='Leading'
        )
    ]
)

[None, None, None, None, None]

In [54]:
def extract_info_from_rss(all_text):



    class TradingEntity(BaseModel):
        """Single trading signal."""
        Date: str = Field(..., description='The publication date of the news')
        name: str = Field(..., description="Sector or company name (exact match from news)")
        note: str = Field("", description="Optional caution/note like '(with caution)'")

    class TradingSignals(BaseModel):
        """Bullish vs Bearish signals from news."""
        bullish: List[TradingEntity] = Field(
            default_factory=list,
            description="Bullish sectors/companies with growth momentum"
        )
        bearish: List[TradingEntity] = Field(
            default_factory=list, 
            description="Bearish sectors/companies with risks/weakness"
        )

    # Prompt - NO {schema} variable needed!
    system_message = SystemMessagePromptTemplate.from_template(
        """You are Ava, elite trader scanning news for actionable signals.

    From the news, extract **BULLISH** (growth/strength) vs **BEARISH** (risks/weakness) sectors/companies.

    Output as JSON array of exact names from news. Use "note" for cautions.

    If NO bullish signals: use empty [] for bullish
    If NO bearish signals: use empty [] for bearish 

    Examples:
    - Bullish: "Private credit sector", note="with caution"
    - Bearish: "European carmakers"

    Rules:
    - 3-8 per list maximum
    - Only signals with clear news evidence
    - Companies + sectors both OK"""
    )

    human_message = HumanMessagePromptTemplate.from_template(
        """News feed:
    {news_feed}"""
    )

    # Create prompt (variables: only {news_feed})
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

    # with_structured_output AUTOMATICALLY injects schema!
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    chain = chat_prompt | llm.with_structured_output(TradingSignals)

    # Run
    # all_text = ". ".join(doc.page_content for doc in docs)
    result = chain.invoke({"news_feed": all_text})

    return result



In [55]:
[rich.print(extract_info_from_rss(cont)) for cont in joined_contents]

TradingSignals(
    bullish=[],
    bearish=[TradingEntity(Date='2025-07-21', name='Chinese companies', note='due to fierce price wars')]
)

TradingSignals(
    bullish=[TradingEntity(Date='2025-07-22', name='solid-state batteries', note='(with caution)')],
    bearish=[
        TradingEntity(Date='2025-07-22', name="China's affluent", note='(pessimistic about the economy)'),
        TradingEntity(Date='2025-07-22', name="Japan's trade position", note='(weaker position in trade talks)')
    ]
)

TradingSignals(
    bullish=[
        TradingEntity(Date='2025-07-23', name='Dongfang Electric', note=''),
        TradingEntity(Date='2025-07-23', name='Tibet Tianlu', note='with caution')
    ],
    bearish=[TradingEntity(Date='2025-07-23', name='SAP', note='flags U.S. trade slowdown')]
)

TradingSignals(
    bullish=[
        TradingEntity(Date='2025-07-24', name='SK Hynix', note='strong AI memory demand'),
        TradingEntity(Date='2025-07-24', name='Deutsche Bank', note='profit beat')
    ],
    bearish=[
        TradingEntity(Date='2025-07-24', name='Tesla', note='loses market share in Europe'),
        TradingEntity(
            Date='2025-07-24',
            name='European automakers',
            note='reported a drop in new car registrations'
        )
    ]
)

TradingSignals(
    bullish=[
        TradingEntity(
            Date='2025-07-25',
            name="U.K.–India 'historic' deal",
            note='set to boost bilateral trade by over $34 billion a year'
        )
    ],
    bearish=[
        TradingEntity(
            Date='2025-07-25',
            name='European firms',
            note='downgraded their guidance due to tariff hits'
        ),
        TradingEntity(
            Date='2025-07-25',
            name='Volkswagen',
            note='cut guidance after taking $1.5 billion hit from U.S. tariffs'
        )
    ]
)

[None, None, None, None, None]

In [56]:
def extract_info_from_rss(all_text):


    class TradingEntity(BaseModel):
        """Detailed trading signal with conviction & catalysts."""
        Date: str = Field(..., description='The publication date of the news')
        name: str = Field(..., description="Exact sector/company name from news")
        
        # Trading conviction (strength of signal)
        conviction: Literal["High", "Medium", "Low"] = Field(
            ..., 
            description="Signal strength: High=strong evidence, Low=emerging/weak"
        )
        
        # Key catalyst driving the signal
        catalyst: str = Field(
            ..., 
            description="Primary news catalyst (1 sentence max)"
        )
        
        # Optional caution/risk
        note: Optional[str] = Field(
            None, 
            description="Risks/cautions like '(regulatory risk)' or '(overbought)'"
        )
        
        # Price action suggestion
        action: Literal["Buy", "Sell", "Watch", "Avoid"] = Field(
            ..., 
            description="Clear trading action"
        )

    class TradingSignals(BaseModel):
        """Detailed bullish/bearish signals."""
        bullish: List[TradingEntity] = Field(min_items=1, max_items=10)
        bearish: List[TradingEntity] = Field(min_items=1, max_items=10)

    # Prompt - NO {schema} variable needed!
    system_message = SystemMessagePromptTemplate.from_template(
        """Ava: Elite trader extracting HIGH-CONVICTION signals from news.

    For each signal provide:
    - name: Exact sector/company from news
    - conviction: High/Medium/Low (evidence strength)  
    - catalyst: 1-sentence key driver
    - action: Buy/Sell/Watch/Avoid
    - note: Risks/cautions (optional)

    Examples:
    - High conviction Buy: "Private credit", catalyst="3.4T→4.9T growth", note="regulatory risk"

    Only clear signals."""
    )

    human_message = HumanMessagePromptTemplate.from_template(
        """News feed:
    {news_feed}"""
    )

    # Create prompt (variables: only {news_feed})
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

    # with_structured_output AUTOMATICALLY injects schema!
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    chain = chat_prompt | llm.with_structured_output(TradingSignals)

    # Run
    # all_text = ". ".join(doc.page_content for doc in docs)
    result = chain.invoke({"news_feed": all_text})

    return result



In [57]:
[rich.print(extract_info_from_rss(cont)) for cont in joined_contents]

ValidationError: 1 validation error for TradingSignals
bullish
  List should have at least 1 item after validation, not 0 [type=too_short, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/too_short